In [1]:
import dirichletcal
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.calibration import CalibratedClassifierCV
from sklearn.isotonic import IsotonicRegression
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import average_precision_score
import numpy as np
import pandas as pd
import pickle
import math
import optuna
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn import preprocessing
from NFLUtils import NFLUtils
nfl_utils = NFLUtils()
from MLBUtils import MLBUtils
mlb_utils = MLBUtils()

%matplotlib inline

import logging
# Set up logging
logging.basicConfig(level=logging.INFO)
# Set pandas display option to show all columns
pd.set_option('display.max_columns', None)

# XGBoost 
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import log_loss
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold, StratifiedKFold, cross_val_score, cross_val_predict
import seaborn as sns # confusion matrix

from dirichletcal.calib.fulldirichlet import FullDirichletCalibrator

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import jax
jax.config.update("jax_platform_name", "cpu")

In [2]:
def set_all_seeds(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Load CSV


In [3]:
df = pd.read_csv("./ttennisData/TTCupSliding50.csv", index_col=False, low_memory=False)
df['H_won'] = df['Total_P1'] > df['Total_P2']
df['Under_74'] = abs(df['Over_74'] - 1)

# Remove the performance set
test_performance_size = 900
test_performance_size = 2000
test_performance_df = df[df.shape[0]-test_performance_size:]
df = df[:df.shape[0]-test_performance_size]
print(df.shape)
df.info()

(77574, 49)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77574 entries, 0 to 77573
Data columns (total 49 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        77574 non-null  int64  
 1   Date              77574 non-null  int64  
 2   Player1           77574 non-null  int64  
 3   Player2           77574 non-null  int64  
 4   Sets_P1           77574 non-null  float64
 5   Sets_P2           77574 non-null  float64
 6   G1_P1             77574 non-null  float64
 7   G1_P2             77574 non-null  float64
 8   G2_P1             77574 non-null  float64
 9   G2_P2             77574 non-null  float64
 10  G3_P1             77574 non-null  float64
 11  G3_P2             77574 non-null  float64
 12  G4_P1             77574 non-null  float64
 13  G4_P2             77574 non-null  float64
 14  G5_P1             77574 non-null  float64
 15  G5_P2             77574 non-null  float64
 16  Win_P1            77574 non-

In [4]:
checkEmpty = df[df["Total_Score"] == 0]
print(checkEmpty.shape)

(0, 49)


In [5]:

# Load data
x_col = [
    'Sets_P1',
    'Sets_P2',
    # 'Total_P1', # Careful re: leakage
    # 'Total_P2', # Careful re: leakage
    'Total_Allowed_P1',
    'Total_Allowed_P2',
    'pythagorean_P1',
    'pythagorean_P2',
    'D_Sets',
    'D_Odds',
#    'No_Odds', # Removed in EDA
    # 'D_Total', # Careful re: leakage
    'D_Total_Allowed'
]



# For H_won
x_col = ['D_Odds', 'pythagorean_P1', 'pythagorean_P2', 'elo_P1', 'elo_P2', 'Total_Allowed_P1', 'G3_P2', 'G1_P1', 'Sets_P2', 'G1_P2', 'Total_Allowed_P2', 'G3_P1', 'G2_P1', 'G2_P2', 'Sets_P1']

# For G45
x_col = ['G4_P2', 'G4_P1', 'D_Odds', 'G5_P1', 'elo_P2', 'Total_Allowed_P2', 'Total_Allowed_P1', 'elo_P1', 'pythagorean_P1', 'Sets_P2', 'G5_P2', 'G1_P1', 'G3_P1', 'G3_P2', 'Sets_P1', 'G2_P2', 'G2_P1', 'pythagorean_P2', 'G1_P2']

# For G5
x_col = ['G5_P1', 'G5_P2', 'Total_Allowed_P2', 'Total_Allowed_P1', 'elo_P2', 'G4_P1', 'G4_P2', 'elo_P1', 'G1_P1', 'pythagorean_P1', 'D_Odds', 'Sets_P1', 'G2_P1', 'G3_P1', 'pythagorean_P2', 'Sets_P2']

# For over 74.5
x_col = ['Total_Allowed_P2', 'Total_Allowed_P1', 'G4_P2', 'G4_P1', 'D_Odds', 'G3_P1', 'Sets_P1', 'G3_P2', 'G5_P1', 'pythagorean_P1', 'G2_P1', 'pythagorean_P2']

y_prediction = 'Over_74'
# y_prediction = 'Under_74'
# y_prediction = 'G5'
# y_prediction = 'G45'
# y_prediction = 'G4'
# y_prediction = 'H_won'

y_col = [y_prediction]
x = df[x_col]
y = df[y_col]

perf_conts_df = test_performance_df[x_col]
perf_y_df = test_performance_df[y_col]


# Create an array of continuous values

Numpy array 'conts' containing stack of each continuous column

In [6]:
conts = np.stack([x[col].values for col in list(x.columns)], 1)
y_col = np.stack([y[col].values for col in list(y.columns)], 1)

perf_conts = np.stack([perf_conts_df[col].values for col in list(perf_conts_df.columns)], 1)
perf_y_col = np.stack([perf_y_df[col].values for col in list(perf_y_df.columns)], 1)

conts_train = conts
y_train = y_col

In [7]:
def convert_probas(probas, local_y_col):
    """
    Using y_col, this converts 2d array of probas into a format that works with the backtest calculation.
    That is, 
    
    probas: 2d array with first element being predicted visitor odds & other being predicted home odds
    y_col: ['H_Won', 'H_start_odds', 'V_start_odds', 'Home_Team_Id', 'Visitor_Team_Id', 'combined']
    """
    # Convert to decimal odds, apply 5% vig estimate
    local_y_col[:,1] = (1 / local_y_col[:,1]) / 1.05
    local_y_col[:,2] = (1 / local_y_col[:,2]) / 1.05
    # probas[:,1] = (1 / probas[:,1])
    # probas[:,0] = (1 / probas[:,0])
    override_req = True
    
    # Set to 0 if < the model's implied probability
    bet_res = []
    for i in range(len(probas)):
        if (probas[i,1] > local_y_col[i,1] or override_req == True) and probas[i,1] > probas[i,0]:
            bet_res.append(probas[i,1]) # Convert back to decimal odds
        elif (probas[i,0] > local_y_col[i,2] or override_req == True) and probas[i,0] > probas[i,1]:
            bet_res.append((-1*probas[i,0])) # Convert back to decimal odds
        else:
            print(f"{probas[i,:]} local ycol = {local_y_col[i,:]}")
            bet_res.append(0)
    return np.array(bet_res)

In [8]:
# conts_train.info()

### Old approach (not based on calibration)

In [9]:
# from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
print(f"BEFORE conts_train: {conts_train.shape}, y_train: {y_col.shape}")
# conts_train, y_col = SMOTE().fit_resample(conts_train, y_col)
# conts_train, y_col = rus.fit_resample(conts_train, y_col)
print(f"AFTER conts_train: {conts_train.shape}, y_train: {y_col.shape}")

BEFORE conts_train: (77574, 12), y_train: (77574, 1)
AFTER conts_train: (77574, 12), y_train: (77574, 1)


In [10]:
# y_col = y_col.reshape(-1, 1)
print(f"AFTER reshape: {conts_train.shape}, y_train: {y_col.shape}")

AFTER reshape: (77574, 12), y_train: (77574, 1)


In [11]:
# print(y_col)

In [12]:
def objective(trial, override_params=None):
    """
    When doing optuna training: trial is defined, override_params left as None
    when recreating model using best parameters (override_params): trial is None, override_params are defined
    """
    undersample_preds = ['G5f', 'G4', 'G45f', 'G54']
    
    is_training = trial != None
    # --- Suggest hyperparameters ---
    if is_training:
        other_eval_metric = ['error', 'auc', 'aucpr', 'logloss']
        if y_prediction in undersample_preds:
            other_eval_metric = ['auc', 'aucpr']
        params = {
            'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 20, log=True),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),  # Add L1 reg
            'gamma': trial.suggest_float('gamma', 0, 5.0),  # Add min split loss
            'other_eval_metric': trial.suggest_categorical('other_eval_metric', other_eval_metric),

            # ONLY FOR G4, G5!!
            # 'scale_pos_weight':trial.suggest_float('scale_pos_weight', 2.3, 2.35, log=True)
        }
        
    else:
        params=override_params

    # For imbalanced datasets (IGNORED BECAUSE OF SMOTE)
    #if y_prediction == 'G5' or y_prediction == 'G4':
    #    scale_weight = sum(y_col == 0) / sum(y_col == 1)
    #    params['scale_pos_weight'] = scale_weight[0] # Under X games / At X games
    
    # Training set to fit XGB
    # Test set to fit Calibrator
    # Holdout set (from test set) to calculate loss
    if is_training:
        loss = []
        kf = KFold(n_splits=9, shuffle=False)
        for train_index, test_index in kf.split(conts_train):
            X_clas = conts_train[train_index]
            X_cal = conts_train[test_index]
    
            y_clas = y_col[train_index].ravel() # ravel=flatten to (n,)
            y_cal = y_col[test_index].ravel()
            
            X_cal, X_holdout, y_cal, y_holdout = train_test_split(X_cal, y_cal, test_size=0.1, shuffle=False)
            # Split a val set from X_clas for early stopping (time-aware)
            X_clas, X_clas_eval, y_clas, y_clas_eval = train_test_split(X_clas, y_clas, test_size=0.1, shuffle=False)
    
            # print(f"X_clas {X_clas.shape} X_cal {X_cal.shape} X_clas_eval {X_clas_eval.shape} X_holdout {X_holdout.shape}")
            
            # Undersample training data
            if y_prediction in undersample_preds:
                # print(f"BEFORE reshape: {X_clas.shape}, y_train: {y_clas.shape}")
                X_clas, y_clas = rus.fit_resample(X_clas, y_clas)
                y_clas = y_clas.reshape(-1,1)
                # print(f"AFTER reshape: {X_clas.shape}, y_train: {y_clas.shape}")
            # Create classifier & fit on test set
            classifier = XGBClassifier(
                objective='binary:logistic',
                # eval_metric='error', # UNCOMMENT FOR Over/Under 74.5
                eval_metric=params['other_eval_metric'],
                early_stopping_rounds=20,
                **{k: v for k, v in params.items() if k.startswith('other_') == False}
            )
            # sample_weight = compute_sample_weight('balanced', y_clas)
            #print(f"sample_weight ${sample_weight}")
            classifier.fit(
                X_clas, y_clas,
                #sample_weight=sample_weight,
                eval_set=[(X_clas_eval, y_clas_eval)],
                verbose=False
            )
            
            # Create calibrator (Note: Ideal to have 5k+ records)
            calibrator = IsotonicRegression(out_of_bounds='clip')
            X_cal = classifier.predict_proba(X_cal)[:,1] # has to be 1D to match y_test
            calibrator.fit(X_cal, y_cal)
    
            # Use holdout set to calculate loss
            X_holdout = classifier.predict_proba(X_holdout)[:,1]
            X_holdout = calibrator.predict(X_holdout)

            # Use AUC-PR if G5 or G4
            holdout_loss = log_loss(y_holdout, X_holdout)
            # if y_prediction == 'G5' or y_prediction == 'G4':
            #    holdout_loss = average_precision_score(y_holdout, X_holdout)
            loss.append(holdout_loss)

        return np.mean(loss)

    
    # ----- When not is_training, train and return classifier, calibrator -----
    # Time-based split: first 90% for clas, last 10% for cal (assumes sorted by time)
    X_clas, X_cal, y_clas, y_cal = train_test_split(conts_train, y_col, test_size=0.1, shuffle=False)
    # Split a val set from X_clas for early stopping (time-aware)
    X_clas, X_clas_eval, y_clas, y_clas_eval = train_test_split(X_clas, y_clas, test_size=0.1, shuffle=False)
    y_clas = y_clas.ravel()
    y_clas_eval = y_clas_eval.ravel()
    y_cal = y_cal.ravel()
    
    # Create classifier
    classifier = XGBClassifier(
        objective='binary:logistic',
        eval_metric=params['other_eval_metric'],
        early_stopping_rounds=20,
        **{k: v for k, v in params.items() if k.startswith('other_') == False}
    )

    # Undersample training data
    if y_prediction in undersample_preds:
        # print(f"BEFORE reshape: {X_clas.shape}, y_train: {y_clas.shape}")
        X_clas, y_clas = rus.fit_resample(X_clas, y_clas)
        y_clas = y_clas.reshape(-1,1)
        # print(f"AFTER reshape: {X_clas.shape}, y_train: {y_clas.shape}")
        
    sample_weight = compute_sample_weight('balanced', y_clas)
    classifier.fit(X_clas, y_clas, sample_weight=sample_weight, eval_set=[(X_clas_eval, y_clas_eval)], verbose=False)

    # Create calibrator, train on classifier predictions
    calibrator = IsotonicRegression(out_of_bounds='clip')
    X_cal = classifier.predict_proba(X_cal)[:,1]
    calibrator.fit(X_cal, y_cal)
    return classifier, calibrator
    

def print_callback(study, trial):
    print(f"Trial {trial.number} finished with value: {trial.value}")
    print(f"Best trial so far: {study.best_trial.number}, value: {study.best_trial.value}")
    
    

In [170]:
study = optuna.create_study(
    direction='minimize',
    pruner=optuna.pruners.HyperbandPruner(min_resource=1, max_resource=1000)
)

study.optimize(objective, n_trials=2000, callbacks=[print_callback])
print("Best trial:")
trial = study.best_trial
print("Value: ", trial.value)
print("Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-10-28 20:29:27,087] A new study created in memory with name: no-name-c3b3ee0f-0c61-46b2-b928-393046788353
[I 2025-10-28 20:29:28,410] Trial 0 finished with value: 0.6924194279990523 and parameters: {'learning_rate': 0.013147610342521824, 'max_depth': 4, 'n_estimators': 735, 'subsample': 0.933552622599009, 'colsample_bytree': 0.6699761366774198, 'min_child_weight': 3, 'reg_lambda': 0.7366185451359458, 'reg_alpha': 2.0595724025572453e-07, 'gamma': 1.0951362529683928, 'other_eval_metric': 'aucpr'}. Best is trial 0 with value: 0.6924194279990523.


Trial 0 finished with value: 0.6924194279990523
Best trial so far: 0, value: 0.6924194279990523


[I 2025-10-28 20:29:44,231] Trial 1 finished with value: 0.6951616360100039 and parameters: {'learning_rate': 0.0032389474096477804, 'max_depth': 9, 'n_estimators': 581, 'subsample': 0.5176012992603908, 'colsample_bytree': 0.8171301689011155, 'min_child_weight': 4, 'reg_lambda': 6.087345564645858, 'reg_alpha': 1.1125715468740487e-06, 'gamma': 0.33491264357824957, 'other_eval_metric': 'logloss'}. Best is trial 0 with value: 0.6924194279990523.


Trial 1 finished with value: 0.6951616360100039
Best trial so far: 0, value: 0.6924194279990523


[I 2025-10-28 20:29:45,779] Trial 2 finished with value: 0.690262426061054 and parameters: {'learning_rate': 0.002388875978803795, 'max_depth': 8, 'n_estimators': 597, 'subsample': 0.7476199232526433, 'colsample_bytree': 0.7452172148373307, 'min_child_weight': 12, 'reg_lambda': 0.08225718179769013, 'reg_alpha': 0.0014286060735183902, 'gamma': 4.275016067875047, 'other_eval_metric': 'auc'}. Best is trial 2 with value: 0.690262426061054.


...

Trial 1997 finished with value: 0.6891804043904526
Best trial so far: 598, value: 0.6885969264344991


[I 2025-10-28 21:12:55,614] Trial 1998 finished with value: 0.6915898317882505 and parameters: {'learning_rate': 0.014889667220389575, 'max_depth': 3, 'n_estimators': 302, 'subsample': 0.7851057474504335, 'colsample_bytree': 0.5221058387385965, 'min_child_weight': 18, 'reg_lambda': 0.02131270211299813, 'reg_alpha': 0.06883729717677399, 'gamma': 2.7400107487477987, 'other_eval_metric': 'auc'}. Best is trial 598 with value: 0.6885969264344991.


Trial 1998 finished with value: 0.6915898317882505
Best trial so far: 598, value: 0.6885969264344991


[I 2025-10-28 21:12:57,008] Trial 1999 finished with value: 0.6921295609291861 and parameters: {'learning_rate': 0.03723240957133098, 'max_depth': 4, 'n_estimators': 324, 'subsample': 0.7507035246393582, 'colsample_bytree': 0.5344908801573565, 'min_child_weight': 19, 'reg_lambda': 0.015246377897384079, 'reg_alpha': 5.585333366022159e-07, 'gamma': 2.9490525295067793, 'other_eval_metric': 'auc'}. Best is trial 598 with value: 0.6885969264344991.


Trial 1999 finished with value: 0.6921295609291861
Best trial so far: 598, value: 0.6885969264344991
Best trial:
Value:  0.6885969264344991
Params: 
    learning_rate: 0.03792849951262246
    max_depth: 3
    n_estimators: 368
    subsample: 0.7745882502364526
    colsample_bytree: 0.5783370786187912
    min_child_weight: 19
    reg_lambda: 0.010214403456232415
    reg_alpha: 6.223306902226365e-08
    gamma: 2.701023147596355
    other_eval_metric: auc


# Over/under 74.5 points
### Test best params & check how calibrated they are

In [13]:
# Trial 260 finished with value: 0.689417728956386
bestParams = {'learning_rate': 0.03959299078566123, 'max_depth': 2, 'n_estimators': 116, 'subsample': 0.8880039974695518, 'colsample_bytree': 0.7735993499483397, 'min_child_weight': 5, 'reg_lambda': 3.956383913116503, 'reg_alpha': 0.004923916196709604, 'gamma': 0.14719743166780305, 'other_eval_metric':'error'}

# Trial 1039 finished with value: 0.6893976627336035 (This is best 10/28/25)
bestParams = {'learning_rate': 0.024741164833444476, 'max_depth': 6, 'n_estimators': 114, 'subsample': 0.7784223531891586, 'colsample_bytree': 0.6247780028956175, 'min_child_weight': 8, 'reg_lambda': 0.3758603208997029, 'reg_alpha': 1.9149256323607524e-05, 'gamma': 0.6465723884007113, 'other_eval_metric':'error'}

# Best trial so far: 598, value: 0.6885969264344991 (earlier odds may be better)
#bestParams =  {'learning_rate': 0.03792849951262246, 'max_depth': 3, 'n_estimators': 368, 'subsample': 0.7745882502364526, 'colsample_bytree': 0.5783370786187912, 'min_child_weight': 19, 'reg_lambda': 0.010214403456232415, 'reg_alpha': 6.223306902226365e-08, 'gamma': 2.701023147596355, 'other_eval_metric': 'auc'}
# Best trial so far: 247, value: 0.6887602022167105 (earlier odds may be better)
# bestParams = {'learning_rate': 0.019121375057154093, 'max_depth': 3, 'n_estimators': 315, 'subsample': 0.7753063635332988, 'colsample_bytree': 0.5380834314337124, 'min_child_weight': 19, 'reg_lambda': 0.017035757489041285, 'reg_alpha': 3.9505847235869564e-07, 'gamma': 2.731862289717825, 'other_eval_metric': 'auc'}

classifier, calibrator = objective(None, bestParams)


# H_won

In [14]:
# Trial 19 finished with value: 0.6768958055818088
bestParams = {'learning_rate': 0.05937265565289764, 'max_depth': 4, 'n_estimators': 962, 'subsample': 0.9765638360100586, 'colsample_bytree': 0.828105890020878, 'min_child_weight': 1, 'reg_lambda': 1.5424681954224457, 'reg_alpha': 0.010763953741291608, 'gamma': 0.7340844790769335, 'other_eval_metric': 'logloss'}

# (No odds) Best trial so far: 12, value: 0.6846107052359698
bestParams =  {'learning_rate': 0.07901247593312617, 'max_depth': 6, 'n_estimators': 980, 'subsample': 0.52238598254258, 'colsample_bytree': 0.6686876677189448, 'min_child_weight': 9, 'reg_lambda': 0.8569450192455178, 'reg_alpha': 1.015742671304129e-06, 'gamma': 0.29732139055797724, 'other_eval_metric': 'error'}

# Best trial so far: 0, value: 0.678127921090242
bestParams = {'learning_rate': 0.002610119419945401, 'max_depth': 7, 'n_estimators': 556, 'subsample': 0.8956572505213283, 'colsample_bytree': 0.7482086886780271, 'min_child_weight': 9, 'reg_lambda': 3.538971364161639, 'reg_alpha': 4.0109507428180954e-07, 'gamma': 1.284790152179896, 'other_eval_metric': 'logloss'}

# Best trial so far: 243, value: 0.6764620882672173
bestParams = {'learning_rate': 0.06552542856853143, 'max_depth': 4, 'n_estimators': 504, 'subsample': 0.9243521642791432, 'colsample_bytree': 0.9681206815300454, 'min_child_weight': 10, 'reg_lambda': 0.040428841065212245, 'reg_alpha': 2.0642270469027124e-07, 'gamma': 3.3662217527056506, 'other_eval_metric': 'error'}

# Best trial so far: 779, value: 0.676456157067713
bestParams = {'learning_rate': 0.05082519468408648, 'max_depth': 4, 'n_estimators': 581, 'subsample': 0.9298260456307847, 'colsample_bytree': 0.9491219872803598, 'min_child_weight': 10, 'reg_lambda': 0.061620009454451764, 'reg_alpha': 6.476257397545602e-07, 'gamma': 3.040396877396999, 'other_eval_metric': 'error'}


classifier, calibrator = objective(None, bestParams)


# G5

In [42]:

bestParams = {'learning_rate': 0.001668893223401683, 'max_depth': 3, 'n_estimators': 282, 'subsample': 0.6121824533078637, 'colsample_bytree': 0.7024085497833387, 'min_child_weight': 5, 'reg_lambda': 0.5161830294412345, 'reg_alpha': 0.00036250386386460807, 'gamma': 0.5349646449008502, 'other_eval_metric':'error'}
# Best trial so far: 426, value: 0.6258810119230529
bestParamns = {'learning_rate': 0.0036011167018495695, 'max_depth': 3, 'n_estimators': 197, 'subsample': 0.6467469679915824, 'colsample_bytree': 0.9997302821616035, 'min_child_weight': 8, 'reg_lambda': 10.067282476830579, 'reg_alpha': 0.001991609177273599, 'gamma': 0.030167521839034227, 'other_eval_metric':'error'}
# Best trial so far: 166, value: 0.6252501968435211
bestParams = {'learning_rate': 0.027266036120905248, 'max_depth': 2, 'n_estimators': 200, 'subsample': 0.7059240984039487, 'colsample_bytree': 0.7240989513964091, 'min_child_weight': 6, 'reg_lambda': 6.510317449049249, 'reg_alpha': 0.006093033379060273, 'gamma': 0.7676383649991234, 'other_eval_metric':'error'}
# Best trial so far: 41, value: 0.627195472800645
#bestParams = {'learning_rate': 0.022198637955711826, 'max_depth': 4, 'n_estimators': 840, 'subsample': 0.6978311818151439, 'colsample_bytree': 0.8913667292100208, 'min_child_weight': 17, 'reg_lambda': 9.993216991868618, 'reg_alpha': 0.17562450749094413, 'gamma': 3.0313640982489645, 'other_eval_metric': 'aucpr'}

#Trial 41 finished with value: 0.6274301292328124
#bestParams = {'learning_rate': 0.0014117009434666905, 'max_depth': 6, 'n_estimators': 504, 'subsample': 0.7631716022434903, 'colsample_bytree': 0.8146818918258596, 'min_child_weight': 14, 'reg_lambda': 2.61626325093829, 'reg_alpha': 1.4018938876036097e-06, 'gamma': 1.8826781662699252, 'other_eval_metric': 'aucpr'}

# After adding params
# Best trial so far: 35, value: 0.6277626037617653
#bestParams = {'learning_rate': 0.010481591170105551, 'max_depth': 10, 'n_estimators': 934, 'subsample': 0.5927564139885113, 'colsample_bytree': 0.8578145346788956, 'min_child_weight': 12, 'reg_lambda': 12.603374199059594, 'reg_alpha': 0.004230970306837592, 'gamma': 4.247732383030352, 'other_eval_metric': 'auc'}

# (H_won params) Best trial so far: 292, value: 0.6264107968655782
# Best trial so far: 444, value: 0.6267165932732717
# bestParams = {'learning_rate': 0.004479686381666177, 'max_depth': 3, 'n_estimators': 535, 'subsample': 0.7732732172099207, 'colsample_bytree': 0.5904808330110781, 'min_child_weight': 1, 'reg_lambda': 0.012661455439150244, 'reg_alpha': 6.46892444779436e-08, 'gamma': 0.1797034230023018, 'other_eval_metric': 'auc', 'scale_pos_weight': 2.3015646015241678}

classifier, calibrator = objective(None, bestParams)



# G4

In [14]:
# Best trial so far: 21, value: 0.69035325703922
bestParams = {'learning_rate': 0.0036972911822452745, 'max_depth': 6, 'n_estimators': 461, 'subsample': 0.837948395479078, 'colsample_bytree': 0.7464564955294609, 'min_child_weight': 9, 'reg_lambda': 1.5015183342270104, 'reg_alpha': 0.008327405442157302, 'gamma': 0.4165157049551762, 'other_eval_metric': 'logloss'}


# BAD Trial 1324 finished with value: 0.66363371914504
# bestParams = {'learning_rate': 0.004196523119054237, 'max_depth': 7, 'n_estimators': 948, 'subsample': 0.5418589299331488, 'colsample_bytree': 0.6109910817939306, 'min_child_weight': 6, 'reg_lambda': 0.12499138337817949, 'reg_alpha': 0.3933365552838541, 'gamma': 3.069263997186816, 'other_eval_metric': 'error'}
# BAD Best trial so far: 1564, value: 0.6635101927157276
# bestParams = {'learning_rate': 0.0022864798714392924, 'max_depth': 7, 'n_estimators': 447, 'subsample': 0.6561306634507178, 'colsample_bytree': 0.6205386352886686, 'min_child_weight': 1, 'reg_lambda': 0.17191287633035035, 'reg_alpha': 0.37850611958819935, 'gamma': 4.424027686126811, 'other_eval_metric': 'error'}

# Trial 5 finished with value: 0.6668067518550506
bestParams = {'learning_rate': 0.29978032406177285, 'max_depth': 9, 'n_estimators': 102, 'subsample': 0.6241784202447722, 'colsample_bytree': 0.5786167862043247, 'min_child_weight': 10, 'reg_lambda': 0.17761598537918127, 'reg_alpha': 0.012857595281312247, 'gamma': 0.027314786005158687, 'other_eval_metric': 'auc'}

# Trial 184 finished with value: 0.6667294256662116
bestParams = {'learning_rate': 0.0027548463523250607, 'max_depth': 9, 'n_estimators': 538, 'subsample': 0.7195716951811234, 'colsample_bytree': 0.5294600322240618, 'min_child_weight': 13, 'reg_lambda': 4.586385852367849, 'reg_alpha': 0.02340998364616693, 'gamma': 4.609922376451254, 'other_eval_metric': 'aucpr'}
bestParams = {'learning_rate': 0.0922281098253217, 'max_depth': 9, 'n_estimators': 183, 'subsample': 0.8486376951337543, 'colsample_bytree': 0.5860672125031787, 'min_child_weight': 17, 'reg_lambda': 1.382584982058371, 'reg_alpha': 4.689260503629778e-07, 'gamma': 2.403968278530627, 'other_eval_metric': 'auc'}
# After SMOTE
# Trial 18 finished with value: 0.6668979600593431

bestParams = {'learning_rate': 0.13304743286015677, 'max_depth': 5, 'n_estimators': 844, 'subsample': 0.9237166179399163, 'colsample_bytree': 0.5650385017853643, 'min_child_weight': 16, 'reg_lambda': 0.1995848855545275, 'reg_alpha': 0.8803236086367877, 'gamma': 4.451798087570364, 'other_eval_metric': 'auc'}
classifier, calibrator = objective(None, bestParams)



# G45

In [33]:
# Best trial so far: 26, value: 0.6031041690958991


# Best trial so far: 189, value: 0.6021270598767348
bestParams = {'learning_rate': 0.06277576241977738, 'max_depth': 3, 'n_estimators': 723, 'subsample': 0.6920574475414543, 'colsample_bytree': 0.7152852499939504, 'min_child_weight': 12, 'reg_lambda': 13.476100594665269, 'reg_alpha': 0.00012886263429678392, 'gamma': 0.6912806584652373, 'other_eval_metric': 'auc'}

# Best trial so far: 2523, value: 0.6021068917261723
bestParams = {'learning_rate': 0.019571701678877305, 'max_depth': 3, 'n_estimators': 524, 'subsample': 0.5163595833889137, 'colsample_bytree': 0.938425232158752, 'min_child_weight': 19, 'reg_lambda': 0.19639593254896393, 'reg_alpha': 3.631358417749485e-06, 'gamma': 2.509164327581109, 'other_eval_metric': 'auc'}

# No SMOTE
bestParams = {'learning_rate': 0.0022373251863992403, 'max_depth': 3, 'n_estimators': 748, 'subsample': 0.9510905446342681, 'colsample_bytree': 0.8181027686655234, 'min_child_weight': 4, 'reg_lambda': 0.015884275778326674, 'reg_alpha': 0.011230742786315313, 'gamma': 4.90491075278029, 'other_eval_metric': 'error'}

classifier, calibrator = objective(None, bestParams)


### Save classifier & calibrator

In [168]:
import pickle
pickle.dump(classifier, open("ttennis_classifier.pkl", 'wb'))
pickle.dump(calibrator, open("ttennis_calibrator.pkl", "wb"))


In [41]:
with open('ttennisData/ttennis_classifierProdOct25.pkl', 'rb') as f:
    classifier = pickle.load(f)

with open('ttennisData/ttennis_calibratorProdOct25.pkl', 'rb') as f:
    calibrator = pickle.load(f)

### Calculate ROC, AUC

In [17]:
from sklearn.metrics import accuracy_score, roc_auc_score

perf_conts_res = classifier.predict_proba(perf_conts)[:,1]
perf_conts_calibrated = calibrator.predict(perf_conts_res)
print(y_prediction)
# Ignore lower bound if predicting 5 games (G5)
include_lower_bound = 0 # Include lower bound
if y_prediction in ['G5', 'G4', 'G45', 'G54']:
    print(y_prediction)
    include_lower_bound = 1.0 # Dont include lower bound
    perf_conts_calibrated = perf_conts_calibrated + 0.17

# IGNORE CALIBRATION
# perf_conts_res = classifier.predict(perf_conts)
perf_conts_calibrated_2d = np.stack([
    1 - perf_conts_calibrated,  # Index 0: P(class=0)
    perf_conts_calibrated       # Index 1: P(class=1)
], axis=1)
perf_conts_calibrated = perf_conts_calibrated_2d

# Filter out where near
print(perf_conts_calibrated.shape)
# print(perf_conts_calibrated)
# Limit usually 0.65-0.7
limit = 0.06
lower_bound = 0.5 - limit - include_lower_bound
upper_bound = 0.5 + limit
# Extract positive class probabilities
positive_probs = perf_conts_calibrated[:, 1]
# positive_probs = perf_conts_calibrated

# Filter out records within ±0.03 of 0.5
mask = (positive_probs < lower_bound) | (positive_probs > upper_bound)
perf_conts_calibrated = perf_conts_calibrated[mask]
perf_y_col_calibrated = perf_y_col[mask]
perf_y_col_calibrated = perf_y_col_calibrated.ravel()
print(perf_y_col_calibrated.shape)


perf_y_eval = np.argmax(perf_conts_calibrated, axis=1)
# print(perf_conts_calibrated)
# print(perf_y_col_calibrated)
print(perf_y_eval.shape)

print('ROC AUC (higher is better): ',  roc_auc_score(perf_y_col_calibrated, perf_conts_calibrated[:,1], multi_class='ovr'), 'Accuracy', accuracy_score(perf_y_col_calibrated, perf_y_eval))
print(f"Raw probs mean: {np.mean(perf_conts_res):.3f}, std: {np.std(perf_conts_res):.3f}")
print(f"Calibrated probs mean: {np.mean(perf_conts_calibrated):.3f}, std: {np.std(perf_conts_calibrated):.3f}")
print(f"Fraction >0.5 (calibrated): {np.mean(perf_conts_calibrated > 0.5):.2%}")
# pcc_tensor = torch.FloatTensor(perf_conts_calibrated[:,0])

# print(perf_conts_calibrated)


Over_74
(2000, 2)
(358,)
(358,)
ROC AUC (higher is better):  0.505253164556962 Accuracy 0.5614525139664804
Raw probs mean: 0.500, std: 0.016
Calibrated probs mean: 0.500, std: 0.115
Fraction >0.5 (calibrated): 50.00%


#### from sklearn.calibration import calibration_curve
from sklearn.metrics import brier_score_loss
import matplotlib.pyplot as plt

# Assuming y_true is true labels and y_prob is predicted probabilities for one class
prob_true, prob_pred = calibration_curve(perf_y_col_calibrated, perf_y_eval, n_bins=10)
plt.plot(prob_pred, prob_true, marker='o', label='Calibration Curve')
plt.plot([0, 1], [0, 1], linestyle='--', label='Perfectly Calibrated')
plt.xlabel('Mean Predicted Probability')
plt.ylabel('Fraction of Positives')
plt.title('Calibration Curve')
plt.legend()
plt.show()
print("Brier Score (lower is better):", brier_score_loss(perf_y_col_calibrated, perf_y_eval))

#### Sanity check

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN
X_resampled, y_resampled = SMOTE().fit_resample(conts_train, y_col)
print(X_resampled.shape)
print(conts_train.shape)
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay


X_train, X_test, y_train, y_test = train_test_split(conts_train, y_col, test_size=0.1, shuffle=False)
sample_weight = compute_sample_weight('balanced', y_train)
model = XGBClassifier(random_state=42, scale_pos_weight=1.6)
model.fit(X_train, y_train, sample_weight=sample_weight)
y_pred = model.predict(X_test)
# Evaluate the model
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
print('Accuracy', accuracy_score(y_test, y_pred))
disp.plot()
plt.show()
# fig, ax = plt.subplots(figsize=(8, 8))
# plot_confusion_matrix(model, X_test, y_test, cmap=plt.cm.Blues, ax=ax);


In [ ]:
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(conts_train, y_col, test_size=0.1, shuffle=False)

xgboster_focal = imb_xgb(special_objective='focal')
CV_focal_booster = GridSearchCV(xgboster_focal, {"focal_gamma":[1.0,1.5,2.0,2.5,3.0]})
CV_focal_booster.fit(X_train, y_train)



